<a href="https://colab.research.google.com/github/Welle-Kim/ML_multicampus/blob/master/General_Titanic_Analysis(A).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 불러오기
import pandas as pd
import numpy as np
import random 

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 머신러닝
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore')

데이터 셋 불러오기
- train.csv : 모델 학습에 사용되는 데이터
- test.csv : 모델 적용 대상이 되는 데이터

In [ ]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
# 모델 적용의 일관성을 위해,
# train과 test를 합한 combine 데이터 셋도 지정
combine=[train_df,test_df]

데이터 변수(Feature) 확인

In [ ]:
# 변수 보기
print(train_df.columns.values)

#### 변수설명
- PassengerId : 각 승객의 고유 번호
- Survived : 생존 여부(종속 변수)
 - 0 = 사망
 - 1 = 생존
- Pclass : 객실 등급 - 승객의 사회적, 경제적 지위
 - 1st = Upper
 - 2nd = Middle
 - 3rd = Lower
- Name : 이름
- Sex : 성별
- Age : 나이
- SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수
- Parch : 동반한 Parent(부모) Child(자식)의 수
- Ticket : 티켓의 고유넘버
- Fare : 티켓의 요금
- Cabin : 객실 번호
- Embarked : 승선한 항
 - C = Cherbourg
 - Q = Queenstown
 - S = Southampton

In [ ]:
# 데이터 미리 보기
train_df.head()

예측의 대상인 object 변수(Label)은 "Survived", 나머지는 설명 변수로 작용됨

In [ ]:
train_df.tail()

In [ ]:
# 훈련 자료 정보
train_df.info()

In [ ]:
# 테스트 자료 정보
test_df.info()

데이터 분석

In [ ]:
# 훈련 자료 int, float 변수 통계치
train_df.describe()

 - 훈련 자료 샘플 수 : 891 (count=891)
 - 훈련 자료 샘플 내 생존율 : 38.4% (mean의 Survived=0.383838)

In [ ]:
# 훈련 자료 object 변수 통계치
train_df.describe(include=['O'])

 - 훈련 자료 남성 수 : 577명 (top의 Sex = male, freq의 Sex = 577)
 - 훈련 자료 가장 많은 승선지 : S, 644명 (top의 Embarked = S, freq의 Emabarked =644)

훈련 자료에서 객실 등급(Pclass)에 따른 생존율 비교

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
#groupby에 as_index를 False로 하면 Pclass를 index로 사용하지 않음
# ascending : 오름차순
# as_index를 True로 하면 Pclass를 index로 사용

 - 객실 등급이 좋을 수록 생존율이 높음

훈련 자료에서 성별(Sex)에 따른 생존율 비교

In [ ]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

 - 여성의 생존율이 남성보다 높음

훈련 자료에서 함께 승선한 형제자매와 배우자 수(SibSp)에 따른 생존율 비교

In [ ]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

훈련 자료에서 함께 승선한 부모와 자식 수(Parch)에 따른 생존율 비교

In [ ]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

 - 동행이 적은 경우, 생존율이 높음

훈련 자료에서 생존 여부(Survived)에 따른 연령(Age) 분포

In [ ]:
# 열(col)을 생존 여부로 나눔
g = sns.FacetGrid(train_df, col='Survived')
# 히스토그램으로 시각화, 연령의 분포를 확인, 히스토그램 bin을 20개로 설정
g.map(plt.hist, 'Age', bins=20)

 - 4세 이하의 유아의 생존율이 높음
 - 15 ~ 25세 승객들의 생존율이 높음

훈련 자료에서 객실 등급(Pclass)과 생존 여부(Survived)에 따른 연령(Age) 분포

In [ ]:
# 열을 생존 여부, 행(row)과 색깔(hue)을 객실 등급으로 나눔, width = height * aspect
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', hue="Pclass", height=2.2, aspect=1.6)

grid.map(plt.hist, 'Age', alpha=.5, bins=20) # 투명도(alpha): 0.5

# 범례 추가
grid.add_legend();

 - 객실 등급이 3등급인 경우, 승객 수는 가장 많고, 생존율도 가장 낮음
 - 객실 등급이 2등급인 유아는 대부분 생존함
 - 객실 등급이 1등급인 경우 생존율이 비교적 높음

훈련자료에서 승선지(Embarked)와 객실 등급(Pclass)에 따른 생존율(Survived)

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6)

# Pointplot으로 시각화, x: 객실 등급, y: 생존 여부, 색깔: 성별, x축 순서: [1, 2, 3], 색깔 순서: [남성, 여성]
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep', order = [1, 2, 3], hue_order = ["male", "female"])

grid.add_legend()

 - 승선지가 C와 Q인 경우, 남성의 티켓 등급이 3등급일 때 2등급보다 생존율이 높을 가능성이 있음

In [ ]:
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)

# 바그래프로 시각화, x: 성별, y: 요금, Error bar: 표시 안 함
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None,order=["male","female"])

grid.add_legend()

 - 승선지가 S또는 C인 경우, 생존한 승객들의 평균 요금이 비교적 높음


## 데이터 전처리

안쓸 변수(Ticket, Cabin) 제거

In [ ]:
print("Before", train_df.shape, test_df.shape)

# 열(axis=1)제거
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]
print("After", train_df.shape, test_df.shape)

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

female에서는 Miss와 Mrs가, male에서는 Master와 Mr가 두드러지게 나타남<br>(Mlle와 Ms 는 Miss의, Ms는 Mrs의 불어식 표현)

나머지는 Rare로 분류

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

Title 변수를 숫자형 변수로 바꿔줌

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

안쓸 변수(Name, PassengerId) 제거

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

성별(Sex) 변수를 숫자 범주형 변수로 바꿔줌

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()


객실 등급(Pclass)과 성별(Sex) 시각화

In [ ]:
g = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.2, aspect=1.6)
g.map(plt.hist, 'Age', alpha=.5, bins=20)
g.add_legend()

 - Pclass, Sex와 Age와의 결합이 2열 3행으로 구성됨

In [ ]:
guess_ages = np.zeros((2,3))
for dbset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dbset[(dbset['Sex'] == i) & \
                                  (dbset['Pclass'] == j+1)]['Age'].dropna()
            # 위에서 guess_ages사이즈를 [2,3]으로 잡아뒀으므로 j의 범위도 이를 따름
            
            age_guess = guess_df.median()

            # age의 random값의 소수점을 .5에 가깝도록 변형
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dbset.loc[ (dbset.Age.isnull()) & (dbset.Sex == i) & (dbset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dbset['Age'] = dbset['Age'].astype(int)

train_df.isnull().sum()

 - Age의 결측치가 채워짐

연령(Age) 변수를 범주형 변수로 바꿔줌

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
# 임의로 5개 그룹을 지정
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

AgeBand를 바탕으로 Age를 범주형 변수로 바꿔준 후, AgeBand변수는 제거

In [ ]:
for dbaset in combine:    
    dbaset.loc[ dbaset['Age'] <= 16, 'Age'] = 0
    dbaset.loc[(dbaset['Age'] > 16) & (dbaset['Age'] <= 32), 'Age'] = 1
    dbaset.loc[(dbaset['Age'] > 32) & (dbaset['Age'] <= 48), 'Age'] = 2
    dbaset.loc[(dbaset['Age'] > 48) & (dbaset['Age'] <= 64), 'Age'] = 3
    dbaset.loc[ dbaset['Age'] > 64, 'Age']
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

SibSp와 Parch를 가족과의 동반여부를 알 수 있는 새로운 변수로 통합

In [ ]:
for dbset in combine:
    dbset['FamilySize'] = dbset['SibSp'] + dbset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

 - FamilySize가 1인 것은 가족과 동반하지 않음을 의미

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

 - 1은 동반X
 - 0은 동반했다는 새로운 변수 IsAlone을 생성

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]
train_df.head()

Age변수와 Pclass를 곱한 Age*Class변수 생성

In [ ]:
for dbset in combine:
    dbset['Age*Class'] = dbset.Age * dbset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

승선지(Ebmarked) 변수를 최빈값으로 대체

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dbset in combine:
    dbset['Embarked'] = dbset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

승선지(Ebmarked) 변수를 범주형 변수로 바꿔줌

In [ ]:
for dbset in combine:
    dbset['Embarked'] = dbset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

요금(Fare) 변수를 중앙값으로 대체

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

요금(Fare)을 숫자 범주형 변수로 바꿔줌

In [ ]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dbset in combine:
    dbset.loc[ dbset['Fare'] <= 7.91, 'Fare'] = 0
    dbset.loc[(dbset['Fare'] > 7.91) & (dbset['Fare'] <= 14.454), 'Fare'] = 1
    dbset.loc[(dbset['Fare'] > 14.454) & (dbset['Fare'] <= 31), 'Fare']   = 2
    dbset.loc[ dbset['Fare'] > 31, 'Fare'] = 3
    dbset['Fare'] = dbset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

In [ ]:
test_df.head(10)

 - 훈련(train) 데이터셋 준비완료

데이터 준비

In [ ]:
# 목적 변수 제거
X_train = train_df.drop("Survived", axis=1)
#목적 변수 적용
Y_train = train_df["Survived"]
#예측 대상 데이터 셋
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

Logistic Regression

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

- 생존 여부 예측에 영향력이 큰 변수는 성별(Sex)

SVC(Support Vector Machines)

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

K-NN(K Nearest Neighberhood)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

Decision Tree

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

Random Forest

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

전체 모델 비교

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

참조: https://computer-science-student.tistory.com/113